# RAG提取文中的氨基酸肽序列

In [2]:
import pandas as pd
import json

#获取文章的pmcid-pmid键值对
dic={'pmcid':[],'pmid':[]}
with open('/mnt/nas/handu/data/anti_inflammation_pubmed/full_text_pmcid_pmid_map.json','r') as f:
    js_list=json.load(f)
    for unit in js_list:
        dic['pmcid'].append(unit['pmcid'])
        dic['pmid'].append(unit['pmid'])

df_ids=pd.DataFrame(dic)


1606    25670897
Name: pmid, dtype: object


In [ ]:
pmid_sel = df_ids[df_ids['pmcid'] == 'PMC4315550']['pmid']
print(pmid_sel)

In [2]:
#获取review标签
with open('/mnt/nas/handu/data/anti_inflammation_pubmed/review_pmid_list.txt','r') as f:
    review_list=f.read().split('\n')
df_ids['is_review']=df_ids['pmid'].apply(lambda x: 1*(x in review_list))
df_ids

,pmcid,pmid,is_review
0,PMC6544583,31193882,0
1,PMC10409771,37553374,0
2,PMC4760456,26660549,0
3,PMC7429751,32795323,0
4,PMC8017164,33815353,0
...,...,...,...
2737,PMC9014016,35444957,0
2738,PMC4484568,26175842,0
2739,PMC4281220,25551573,0
2740,PMC4671727,26642048,0


In [3]:
import os
import sys
sys.path.append(os.path.abspath('/home/qcdong/codes/llamaIndex'))
from utils_llama_index.llm_loader import load_hf_llm
from llama_index.core import PromptTemplate, Settings


os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["LLAMA_INDEX_CACHE_DIR"] = "/mnt/nas1/models/llama_index_cache"
# Settings.llm = load_hf_llm()
Settings.embed_model = "local:/mnt/nas1/models/BAAI/bge-small-en-v1.5"


/home/qcdong/anaconda3/envs/llaIdx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 混合索引引擎

In [4]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.core.schema import BaseNode

from typing import List

In [5]:
from utils_llama_index.retriever_util import CustomRetriever, AutoMergingRetrieverMixedHierarchicalNodes


## 批量文章挖掘seq

In [6]:
import json
import os
import re
from llama_index.core.schema import Document
#os.chdir('/home/cymei/nlp_task/anti_inflammation')

# 文本载入函数。由于此次任务是寻找seq序列，和之前seq序列判断任务不同，暂时不考虑文本预处理
def load_doc(file_name):
    with open(file_name,'r') as f:
        js_file=json.load(f)
        js_text=js_file['text']

    return Document(text=js_text)

In [23]:
# check data
from pathlib import Path

for i, row in df_ids.iterrows():
    pmc_file = f'/mnt/nas/handu/data/anti_inflammation_pubmed/pmc_parse/{row["pmcid"]}.json'
    if not Path(pmc_file).exists():
        continue
    with open(pmc_file,'r') as f:
        js_file=json.load(f)
        text=js_file['text']
    target_txt = 'in ALX/FPR2'
    if target_txt in text:
        print(row['pmcid'])
        i = text.find(target_txt)
        while i > -1:
            print(text[i: i+200])
            i = text.find(target_txt, i + 1)
        print()
"""  
PMC4315550
HIV-Tat (CGGGGYGRKKRRQRRR) peptides were synthesized in the solid phase using fluorenylmethoxycarbonyl protected amino acids following procedures previously described.37,38 The polyglycine sequence is
POD (CGGG[ARKKAAKA]4) and HIV-Tat (CGGGGYGRKKRRQRRR) peptides were synthesized in the solid phase using fluorenylmethoxycarbonyl protected amino acids following procedures previously described.37,38 T
"""
print()

PMC7429751
in ALX/FPR2 expression after SAH over time, as shown in Fig. 1b, the expression of ALX/FPR2 protein in the rat brain significantly increased from 24 h to 3 days before decreasing on the 7th day after 

PMC8096073
in ALX/FPR2 expression was considered to be mainly because of indomethacin-promoted infiltration of inflammatory cells into the small intestinal tissue. Furthermore, indomethacin-induced small intesti

PMC8279484
in ALX/FPR2 expressing cells as a resultof stimulation with ATP (1 μM) or Wp (2 nM). Importantly, ATPstimulated Ca2+ mobilization in wild-type HEK-293 cells,whereas the ALX/FPR2 ligands (sLXms, Wp, LX

PMC4827125
in ALX/FPR2−/− mice treated with LPS. However, previous reports in other disease models have shown that the degree of immune cell infiltration is organ-specific and dependent on time course of infecti
in ALX/FPR2−/− mice treated with LPS as compared to C57BL/6 mice, while no upregulation of pro-inflammatory cytokines was observed in animals treated

In [29]:
import os
from IPython.display import Markdown, display
import time

seed = 0
# 从review和非review样本中随机各抽取5个进行尝试
df_test = pd.concat(
    [
        # df_ids[df_ids['pmcid'] == 'PMC4827125'],
        df_ids[df_ids['pmcid'] == 'PMC4315550'],
        df_ids[df_ids["is_review"] == 0].sample(5, random_state=seed),
        df_ids[df_ids["is_review"] == 1].sample(5, random_state=seed),
    ]
)
print(
    df_ids[df_ids["is_review"] == 0].shape,
    df_ids[df_ids["is_review"] == 1].shape,
)
# 记录模型回答以及支持观点的段落
df_test["model_response"] = ""
df_test["supportive_context"] = ""

pmc_ids_test_list = df_test[
    "pmcid"
].unique()  # df_seq['pmcid'].unique()  PMC2408570  PMC6554247  PMC9786789  PMC6544583
print(len(pmc_ids_test_list))
print(pmc_ids_test_list)

(2366, 3) (376, 3)
11
['PMC4315550' 'PMC7930508' 'PMC2828392' 'PMC4803614' 'PMC4946119'
 'PMC10104456' 'PMC10838977' 'PMC4138593' 'PMC10376369' 'PMC2975941'
 'PMC9025908']


In [30]:
from utils_llama_index.api_client import query_with_nodes

for pmc_id in pmc_ids_test_list:
    file_loc = "/mnt/nas/handu/data/anti_inflammation_pubmed/pmc_parse/{}.json".format(
        pmc_id
    )
    doc_text = load_doc(file_loc)

    # 使用llama-index建立查询引擎
    summarier = AutoMergingRetrieverMixedHierarchicalNodes([doc_text])

    # 设置检索关键词
    summarier.keywords = ["peptide", "anti-inflammatory"]
    retriever = summarier.hierarchy_node_index(
        similarity_top_k=10, max_num_keyword_nodes=4
    )

    # query中的问题
    # 目前该prompt可以引导模型在部分文字中抓取一些seq，但模型无法判断是完整的肽序列还是片段
    question = """
        Please find out all the peptides sequence firstly (motif sequence is not included). The target sequence should only consist of different kinds of peptide.
        Finally, plesae do analysis and explain the reason first, then list all peptides sequences that have the ability of anti-inflammation line by line.
        Please focuse on the given context and don't use prior knowledge. Please return the top 3 most supportive context number.
"""
    question = """
List all the different peptide sequences which is composed by amino acids (protein names are excluded) that have the anti-inflammation ability, line by line. If there are not peptide sequences, reply no peptide sequences are found from the context. Then explain the reason in the next lines.
Finally, return the top 3 most supportive context number.
"""
    # Make sure the target sequence numbers are specific about anti-inflammation and don't make any guesses.
    # Please return the most supportive context number.

    # retriever寻找目标节点
    response_nodes = retriever.retrieve(question)
    response = query_with_nodes(question, response_nodes)

    print(f"--------{pmc_id}--------response_nodes num:{len(response_nodes)}--------")
    display(Markdown(f"<b>{response}</b>"))

    # 根据模型回答找出支持的文本片段
    context_list = re.findall("Context[0-9]+", response)
    context_nums = set(
        [int(re.sub("Context", "", context)) for context in context_list]
    )
    condition = df_test["pmcid"] == pmc_id

    df_test.loc[condition, "model_response"] = response
    df_test.loc[condition, "supportive_context"] = "\n".join(
        [
            "Context{}:\n".format(index) + response_nodes[index].text
            for index in context_nums
            if (index < len(response_nodes))
        ]
    )
    break

> Merging 1 nodes into parent node.
> Parent node id: cc6f46f1-5cf8-4e46-ac5a-41e5e9236468.
> Parent node text: The reaction was carried out using N-diisopropylcarbodiimide and 1-hydroxybenzotriazole in DMF me...

> Merging 1 nodes into parent node.
> Parent node id: 2db8684d-7714-42aa-bd70-115769490257.
> Parent node text: Unreacted POD and HIV-Tat were quantified using Ellman’s assay.44 A solution of 0.1–0.2 μmol of t...

> Merging 1 nodes into parent node.
> Parent node id: e40bf989-493c-4000-a8f4-ce4f96b4ab88.
> Parent node text: These results indicate that a slight drug association exists when the peptides are within the pol...

--------PMC4315550--------response_nodes num:10--------


<b>The different peptide sequences with anti-inflammatory ability mentioned in the context are:

1. POD (CGGG[ARKKAAKA]4): This peptide sequence is mentioned in Context 3, where it is described as being synthesized in the solid phase using fluorenylmethoxycarbonyl protected amino acids. The POD peptide is designed for ocular delivery and has been shown to have anti-inflammatory effects, as discussed in Context 0.

2. HIV-Tat (CGGGGYGRKKRRQRRR): This peptide sequence is also mentioned in Context 3, where it is described as being synthesized in the solid phase using fluorenylmethoxycarbonyl protected amino acids. Although not explicitly mentioned as having anti-inflammatory effects, the HIV-Tat peptide has been studied for its ability to penetrate cells and could potentially be used for drug delivery purposes, which may include anti-inflammatory applications.

The reason these two peptide sequences are considered to have anti-inflammatory ability is due to their association with the development and prevention of ocular inflammation, as discussed in Context 0. The POD peptide, in particular, has been shown to have a significant anti-inflammatory effect in preventing ocular inflammation induced by SA.

The top 3 most supportive context numbers for this query are:

1. Context 0: This context directly discusses the anti-inflammatory effect of the POD peptide in preventing ocular inflammation induced by SA, providing evidence of its anti-inflammatory ability.
2. Context 3: This context provides information on the synthesis of the POD and HIV-Tat peptides, which are both mentioned in Context 0 as having potential anti-inflammatory effects.
3. Context 9: This context discusses the conjugation of Rho-peptides with PLGA-PEG-maleimide to create NPs with potential anti-inflammatory properties, which is related to the development and application of peptides like POD and HIV-Tat.</b>

In [13]:
df_test

,pmcid,pmid,is_review,model_response,supportive_context
1606,PMC4315550,25670897,0,"After analyzing the given contexts, I have fou...",Context0:\nThese results indicate that a sligh...
1911,PMC4827125,27064029,0,"Based on the provided context, there are no pe...",Context0:\nThe goal of this study was to deter...
257,PMC3232377,22059987,0,"Without the provided contents, I am unable to ...",
2024,PMC2902675,20631879,0,"After reviewing the provided contexts, I canno...",Context2:\nIn all of the experiments described...
425,PMC9686674,36421317,0,"Based on the provided context, here are the se...",Context0:\nIt was observed that WIKE-14 can ex...
2320,PMC10230051,37266425,1,"Based on the provided context, there are sever...",Context1:\nActivation of the GLP-1 pathways de...
793,PMC4216654,25338737,1,"Based on the given context, there are several ...",Context0:\n[42]Inhibts glucagon release by a m...
752,PMC8772713,35052724,1,"After analyzing the provided context, I have f...",Context1:\nThe LEAPS approach derives benefits...
1206,PMC9394722,36003669,1,"Based on the given context, there are several ...",Context1:\nThe so-called ‘Cytokine Storm’ that...
225,PMC8413167,34105114,1,"After reviewing the provided context, I have f...",Context0:\nIt seems that the dual properties m...


In [45]:
df_test.to_csv('./10patents_example.csv',index=False)

In [27]:
from llama_index.response.notebook_utils import display_source_node
for node in response_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** bc7b9bc5-089c-492d-b968-96c72a8e391f<br>**Similarity:** 0.8580143326929651<br>**Text:** In fact, a number of current TCM prescriptions use epimedium as the main ingredient or monarch such as the Xianling Gubao capsule, Er-Xian Decoction and Xian-Zhen Decoction. Other ingredients such as psoralea and drynaria rhizome serve as ministers or assistants to the monarch ingredient and are also included in these formulations.38      In this study, we present the development of a minimal TCM formula primarily containing two ingredients, epimedium and drynaria. Epimedium was chosen as the main or monarch ingredient partially because of its TCM warming properties but also because of its in vitro biological activities. Drynaria was chosen as a minister ingredient to the monarch to further strengthen the kidney and to support CRE activation and possible osteogenic activities. The other ingredients such as danshen were considered as assistants for invigorating the spleen and replenishment of qi as well as their anti-inflammatory activity.<br>

--------


**Node ID:** 67fb29a4-ade9-4616-95dd-e561678a8abb<br>**Similarity:** 0.8572586696340335<br>**Text:** LTD and E2 was from Bayer Pharmaceutical Co. Ltd. (Guangzhou, CN).        Vectors pGL4.29[luc2P/CRE/Hygro], pGL4.26 [luc2/minP/Hygro], pGL4.17 [luc2/Neo], pGL4.35 [luc2P/9XGAL4UAS/Hygro], pBIND-ERα-LBD (residues 299-595), and Fugene 6 and Fugene HD were purchased from Promega Inc. (Madison, WI). Vector pNF-κβ-Luc was from Clontech Laboratories (Mountain View, CA). Vectors pCMV6-XL4 ER β (Cat# SC119216), and pCMV6-NEO were purchased from Origene Technologies (Rockville, MD). Vector pCR BluntII TOPO ER α (Cat# MHS6278-211691051) was from Dharmacon (Lafayette, CO).<br>

--------


**Node ID:** 04a36885-efdb-4092-aa83-bf8bb589b0b1<br>**Similarity:** 0.8518454004028271<br>**Text:** 4                    3.3        Anti-inflammatory activity determination        Pretreatment of A549 cells containing the NF-κβ activation reporter construct with either danshen, epimedium, or milkvetch extracts (40 μg/mL) inhibited IL-1β (100 pg/mL) activation of NF-κβ (Fig. 5). The other extracts did not demonstrate appreciable anti-inflammatory activity. The anti-inflammatory activity did not correlate to CRE activation. Drynaria did not inhibit NF-κβ activation by IL-1β despite activating CRE. Bay 11-7082 (12.5 μM) was used as a control inhibitor of IL-1β activated NF-κβ.34Fig. 5Effect of selected TCM plant extracts on the inhibition of NF-κβ activation by IL-1β. Plant extracts were resuspended in DMSO and added at a final concentration of 40 μg/mL to A549 cells stably transfected with an NF-κβ responsive luciferase reporter construct. Cells were stimulated with IL-1β (100 pg/mL). Control treatments were IL-1β only and Bay11 (B; 12.5 μM). Cells were treated for 6 h. Extract abbreviations were as follows: An, Angelica; As, Asiatic Cornelian Cherry; D, Danshen; Dr, Drynaria; DS, Dodder Seed; E, Epimedium, Eu, Eucommia; GP, Glossy Privet; IB, Indian Bread; M, Milkvetch; MC, Medicinal Cyathula; P, Psoralea: R, Rehmannia; TA, Two-toothed Achyranthes; WB, Wolfberry root bark; and Y, Common Yan Rhizome. Data are represented as mean ± SD of relative luminescence units (RLUs (n = 3); #p < 0.0001 compared to untreated (Unt) controls).Fig. 5                    3.4        Activation of bone specific genes Runx2 and Bmp-4        For subsequent evaluations, epimedium and drynaria were selected as companion ingredients and tested in additional bone related assays. Epimedium was selected as the primary (or monarch) ingredient for four reasons: 1) it activates the full-length ERα and ERβ (Fig.<br>

...

--------


**Node ID:** 9a02eaa0-5f93-4579-a089-3ac3136e5230<br>**Similarity:** 0.8399784484138657<br>**Text:** Epimedium and drynaria activated these targets despite the presence of estrogenic materials, phenol-red and FBS.49 We suggest that epimedium and drynaria may be upregulating these genes through their activation of CRE. Fsk, a adenylate cyclase activator and epimedium have previously been shown to upregulate CRE-related responses.44 Drynaria has not been shown previously to activate CRE.      Osteoclast development was inhibited by the epimedium and drynaria combination in a dose-dependent manner and the addition of a third ingredient, either danshen, angelica or wolfberry augmented the inhibition at selected doses. Previously, epimedium alone has been shown to inhibit NF-κβ activation as well as to stimulate p38 MAPK to inhibit NF-κβ during osteoclast differentiation.45, 46, 47, 48 The NF-κβ inhibition by epimedium does not appear to be the result of either CRE activation and/or ER activation50, 51, 52 because other extracts that activate these pathways (drynaria and psoralea, respectively) do not inhibit NF-κβ activation.<br>

--------


**Node ID:** 2ebcb2ed-9bae-4a77-a7a7-3be231763a1f<br>**Similarity:** 0.8328065251198418<br>**Text:** The ERβ-LBD (residues 227-530) was PCR amplified from full-length human ERβ (SC119216, OriGene Technologies (Rockville, MD)) using forward and reverse primers 5′-CGATCGTTCGCCTTGTGCGGAGACAGAGAAGTGCC-3′ and 5′-TCTAGATTACTGAGACTGTGGGTTCTGGGAGCCCTC-3′, incorporating PvuI and XbaI sites at the 5′ and 3′ ends, respectively. The ERβ-LBD fragment was then ligated into the pBIND3 PvuI/XbaI site forming a Gal4- ERβ -LBD fusion protein.                            2.5        Cell culture and stable cell lines        Chinese hamster ovary (CHO-K1), MC3T3-E1 subclone 14, HEK293T/17, SK-N-SH, and A549 were purchased from ATCC (Manasas, VA) and grown in in DMEM or αMEM or DMEM or EMEM or F12K media, respectively.<br>

--------
